In [3]:
# 1. INITIALIZE HAIL (Standard AoU Setup)
hl.init(default_reference='GRCh38')

/opt/conda/lib/python3.10/site-packages/hail/context.py:350: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Running on Apache Spark version 3.5.3
SparkUI available at http://all-of-us-27410-m.us-central1-a.c.terra-vpc-sc-f223a39d.internal:35713
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /home/jupyter/workspaces/asdproject/hail-20251207-2250-0.2.134-952ae203dbbe.log
2025-12-07 23:31:41.130 Hail: INFO: merging 13445 fi

In [4]:
bucket = os.getenv("WORKSPACE_BUCKET")
wgs_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH") 

asd_gene_target_list = {
    "SHANK3", "SCN2A", "MECP2", "CHD8", "SYNGAP1", "NRXN1", "PTEN", "SCN1A", "CNTNAP2", "ARID1B", 
    "ADNP", "DYRK1A", "GRIN2B", "CHD2", "FOXP1", "ANKRD11", "STXBP1", "AUTS2", "TCF4", "FMR1", 
    "GRIN2A", "POGZ", "PCDH19", "KCNQ2", "SCN8A", "TSC2", "CACNA1C", "CDKL5", "FOXG1", "RELN", 
    "DDX3X", "SLC6A1", "ASXL3", "SHANK2", "FOXP2", "IQSEC2", "MEF2C", "MED13L", "NLGN3", "KMT2A", 
    "TRIO", "DMD", "UBE3A", "CHD7", "SATB2", "GABRB3", "KDM5C", "OXTR", "SETD5", "CACNA1A", 
    "MBD5", "TBR1", "NF1", "ASH1L", "CTNNB1", "EHMT1", "KMT2C", "MYT1L", "RBFOX1", "NLGN4X", 
    "TRIP12", "CREBBP", "MET", "ANK2", "CUL3", "HNRNPU", "PPP2R5D", "DYNC1H1", "NEXMIF", "NSD1", 
    "RAI1", "ATRX", "SETBP1", "TSC1", "DEAF1", "DISC1", "ANK3", "CACNA1H", "DNMT3A", "KAT6A", 
    "MTOR", "TCF20", "TTN", "AHDC1", "BCL11A", "CACNA1D", "GRIN1", "KCNMA1", "TBL1XR1", "WAC", 
    "KCNB1", "PTPN11", "VPS13B", "IL1RAPL1", "KMT5B", "NAA15", "ZBTB20", "CACNA1G", "CASK", "CNTN4", 
    "EP300", "NR2F1", "NRXN3", "SETD2", "SLC6A4", "SMARCA2", "HUWE1", "MAGEL2", "PHIP", "SYN1", 
    "CTCF", "DPP6", "KDM5B", "NBEA", "ARX", "KDM6B", "SON", "ATP1A3", "DEPDC5", "EN2", 
    "SETD1B", "SMARCA4", "WDFY3", "AVPR1A", "CDK13", "CHD3", "DLG4", "EBF3", "RORA", "SLC25A12", 
    "SLC6A8", "SOX5", "TLK2", "TRAPPC9", "ASTN2", "CACNA1E", "CNOT3", "CSMD1", "CYFIP1", "HECW2", 
    "KMT2E", "MACROD2", "MTHFR", "SYNE1", "AFF2", "EEF1A2", "MBOAT7", "PTCHD1", "SLC9A6", "AHI1", 
    "BRAF", "GRIK2", "NFIX", "NLGN1", "PHF21A", "CSNK2A1", "DSCAM", "GIGYF1", "HNRNPH2", "ITGB3", 
    "KCNQ3", "OPHN1", "RERE", "ARID2", "CC2D1A", "CTNNA3", "DHCR7", "LRP2", "LZTR1", "MAOA", 
    "MED13", "NIPBL", "PACS1", "RPS6KA3", "SEMA5A", "SMARCC2", "TANC2", "TNRC6B", "TRRAP", "USP9X", 
    "ALDH5A1", "ARHGEF9", "CHAMP1", "DLGAP2", "ITPR1", "KIRREL3", "MCPH1", "PRKN", "SMC1A", "UPF3B", 
    "AGO1", "GRM7", "HDAC4", "IRF2BPL", "JARID2", "KATNAL2", "NFIA", "NRXN2", "RAB39B", "SHANK1", 
    "SPEN", "SRCAP", "ZNF462", "ATP2B2", "CIC", "CNKSR2", "CNTN6", "CTNND2", "FBN1", "GLRA2", 
    "GRIA3", "KIF1A", "PHF8", "RAPGEF4", "SIN3A", "SPAST", "TAF1", "ZMYND11", "C12orf57", "CACNA2D3", 
    "CHRNA7", "CLCN4", "CUX2", "DPYD", "DPYSL2", "FBXO11", "HIVEP2", "IMMP2L", "KDM6A", "KIF5C", 
    "NCKAP1", "NLGN2", "NR4A2", "SLC6A3", "SLC9A9", "USP7", "WWOX", "ACTL6B", "BIRC6", "BRSK2", 
    "CELF4", "CNTNAP4", "CNTNAP5", "CTTNBP2", "DLG2", "GABBR2", "GRIA2", "GRIP1", "HOXA1", "LAMA1", 
    "RORB", "SETD1A", "SLC1A1", "SMC3", "YWHAZ", "ZNF804A", "ABCA13", "ARNT2", "ASPM", "CAMK2A", 
    "CEP290", "CUX1", "EIF4E", "GRIA1", "HCN1", "KCNC2", "KCND3", "KCNJ10", "KLF7", "MAP1B", 
    "NSD2", "NTRK3", "QRICH1", "RNU4-2", "SMAD4", "SRRM2", "TRPM3", "YWHAG", "ZNF292", "CACNA1B", 
    "CAMK2B", "CDK5RAP2", "CHD1", "DIP2A", "DPP10", "FAT1", "HERC1", "KANK1", "KAT6B", "MEIS2", 
    "NAV2", "PACS2", "PCDH15", "POLR3A", "PPP3CA", "PTPRD", "RPL10", "SCN3A", "SCN9A", "SNAP25", 
    "SPTBN1", "TAOK1", "TCF7L2", "ZBTB18", "ZFHX3", "ACY1", "ADCY5", "BRD4", "CADPS2", "CDC42BPB", 
    "CDH8", "CNOT1", "CSNK2B", "DHX30", "DLGAP1", "DLGAP3", "DMPK", "EP400", "FLNA", "FRMPD4", 
    "GABRG3", "HACE1", "HCFC1", "HDAC8", "HEPACAM", "HERC2", "HIVEP3", "HNRNPK", "INTS1", "KANSL1", 
    "KDM2B", "MSRA", "NAA10", "NCOR1", "MSNP1AS", "PCCA", "PCDH9", "POLR2A", "RFX3", "RIMS1", 
    "UBR5", "UNC80", "WDFY4", "ZMIZ1", "BSN", "CACNA1I", "CACNB2", "DNM1", "DOCK4", "DOCK8", 
    "DST", "FEZF2", "FHIT", "GABRG2", "GRB10", "GRM5", "GTF2I", "HECTD4", "HRAS", "KCNA2", 
    "KMT2B", "LRP1", "MACF1", "MARK1", "MRTFB", "NACC1", "NR3C2", "PAH", "PCCB", "PCDH10", 
    "PCLO", "PPM1D", "PRPF8", "PRUNE2", "PTPRT", "ST8SIA2", "SYT1", "USH2A", "USP24", "XPC", 
    "XPO1", "ACTB", "ARHGAP32", "ASMT", "BCL11B", "CACNA1F", "CDH10", "CHKB", "CNTN5", "CSMD3", 
    "DLX2", "DNAH10", "EPPK1", "GPHN", "GRID2", "H1-4", "HLA-DRB1", "ITSN1", "KCNC1", "KCNH7", 
    "LAMB1", "LRBA", "MAPK8IP3", "MYCBP2", "NOTCH1", "NRP2", "NTRK2", "OCRL", "PIK3CA", "PIK3R2", 
    "PLXNA3", "PRR12", "PTPRC", "RAC1", "RANBP17", "ROBO2", "SBF1", "SLC1A2", "SNX14", "STAG1", 
    "TAOK2", "TMLHE", "TRAF7", "UNC13A", "WDR26", "YY1", "ZNF536", "ABL2", "ADGRB1", "AGAP1", 
    "AKAP9", "AP2M1", "APBA2", "ADRB2", "AR", "ATP10A", "ATP1A1", "BCKDK", "BCORL1", "BRCA2", 
    "BRWD3", "CACNA2D1", "CADM1", "CASZ1", "CCDC88C", "CD38", "DLX6", "DMXL2", "DNAH17", "DNAH3", 
    "EPHB1", "GABRA4", "GIGYF2", "GLO1", "GNAS", "GRIK5", "HLA-A", "ICA1", "JMJD1C", "KCND2", 
    "KDM2A", "MDGA2", "MIB1", "MIR137", "MUC4", "MYO16", "NINL", "PARD3B", "PAX5", "PER1", 
    "PKD1", "POMGNT1", "SACS", "SGSH", "SLC12A5", "SLC4A10", "SLITRK5", "SND1", "SUPT16H", "SYNCRIP", 
    "TBC1D5", "TECTA", "TRPC5", "TSPAN7", "UNC79", "UPF2", "VDR", "ZMYM2", "ZNF559", "ADSL", 
    "AGO3", "ARID1A", "BAZ2B", "BTAF1", "AVPR1B", "CASKIN1", "CDH11", "CEP135", "COL12A1", "CRMP1", 
    "CSDE1", "CSMD2", "CTR9", "CUL7", "CYFIP2", "DRD3", "EFR3A", "EPHB2", "ERBIN", "ESR2", 
    "ESRRB", "FRYL", "GPC5", "HSD11B1", "HTR3A", "KCTD13", "KPTN", "MARK2", "MAST3", "MBD1", 
    "MEGF11", "MFRP", "MSL3", "MYH10", "MYH9", "NTRK1", "OTUD7A", "PAX6", "PBX1", "PLCB1", 
    "PPP2R1B", "PRKDC", "PRODH", "PTK7", "PXDN", "PTPRS", "SKI", "TAF6", "TLN2", "TNPO3", 
    "TRAPPC6B", "WNK3", "ZMYM3", "ZMYND8", "ABAT", "ABCA2", "ADA", "AGMO", "ANKS1B", "AP1S2", 
    "ARHGEF10", "ARHGEF2", "CAPN12", "CDH2", "CEP41", "CERT1", "CGNL1", "CHD4", "CHD9", "CHM", 
    "CLTCL1", "CNR1", "CPSF7", "CSNK1E", "DDC", "DIP2C", "EPHA1", "FABP5", "GPC6", "HNRNPR", 
    "HS3ST5", "HTR3C", "IGF1", "IL1R2", "KAT2B", "MED12L", "MEGF10", "METTL14", "MINK1", "MYO5A", 
    "MYO9B", "NFIB", "NPAS3", "NTNG1", "PAK1", "PC", "PCDHA5", "PDE3B", "PDHA1", "PER2", 
    "PHF2", "PLAA", "PPFIA3", "PRICKLE2", "PRKAR1B", "PRKCB", "PTBP2", "PTGS2", "RAB2A", "RPH3A", 
    "SCAF4", "SEZ6L2", "SGSM3", "SIK1", "SLC9A1", "SLITRK2", "STXBP5", "TERF2", "TET2", "TET3", 
    "TOP2B", "TOP3B", "TRIM23", "TRPM7", "TSHZ3", "TSPOAP1", "U2AF2", "ZSWIM6", "ACHE", "ACTN4", 
    "AGAP2", "AHNAK", "ALDH1A3", "ALDH1L1", "ANKRD17", "ADORA2A", "ARHGAP5", "CA6", "CADM2", "CAPRIN1", 
    "CCNK", "CDH22", "CDH9", "CDK19", "CHRM3", "CMIP", "CNGB3", "CNTNAP3", "CYP27A1", "DAGLA", 
    "DDHD2", "DDX53", "DRD2", "DUSP15", "EIF3F", "ELAVL2", "ELP2", "ELP4", "EXT1", "G3BP2", 
    "GABRB2", "GABRA2", "GALNT10", "GALNT14", "GNAI1", "GNB2", "GRID1", "GRIK3", "GRIK4", "H3-3B", 
    "HECTD1", "IKZF1", "IQGAP3", "ITGA8", "KIF13B", "KIF14", "LAS1L", "LRFN2", "LRFN5", "LRRC1", 
    "MAOB", "MBD4", "MSL2", "MTSS2", "MYH4", "NPFFR2", "OFD1", "PAPOLG", "PATJ", "PCDHA7", 
    "PHF14", "PHLPP1", "PITX1", "PLPPR4", "PLXNA4", "PNPLA7", "POLA2", "PRICKLE1", "PRKCA", "PRKD1", 
    "PSD3", "PSMD12", "PTPRB", "RAD21", "RALA", "RIMS2", "RLIM", "RSRC1", "SCN4A", "SET", 
    "SIN3B", "SLC22A9", "SLC23A1", "SNTG2", "SORCS3", "SOS 2", "STX1A", "SYN2", "TBC1D31", "TBX1", 
    "TSPYL2", "UBE3C", "USP27X", "USP45", "YTHDC2", "ZC3H4", "ZFYVE26", "ABCA7", "AGTR2", "ATP9A", 
    "BAIAP2L1", "BBS4", "BST1", "C4B", "CACNG2", "CAMK4", "CCDC91", "CDH13", "CLN8", "CLTC", 
    "CNTN3", "DCC", "DIXDC1", "DLG1", "DENND2B", "DLL1", "DPP4", "DOT1L", "DVL3", "ELAVL3", 
    "EPC2", "FBXO40", "GALNT13", "GALNT2", "GLI3", "GPR85", "GPX1", "GRK4", "H4C5", "HDLBP", 
    "HOMER1", "HTR1B", "ILF2", "INTS6", "KCNH1", "KCNH5", "KCNS3", "KDM3B", "KDM4B", "KIAA1586", 
    "LMTK3", "MAGEC3", "MAPK3", "MTF1", "MUC12", "MYO1E", "MYO5C", "NAV3", "NEGR1", "NFE2L3", 
    "NIPA1", "NLGN4Y", "NRCAM", "NSMCE3", "NTNG2", "NUAK1", "NUP133", "NXPH1", "P2RX5", "PABPC1", 
    "PAK2", "PCDHA10", "PCDHA11", "PCDHA2", "PCDHA4", "PCM1", "PDK2", "PDZD8", "PPFIA1", "PREX1", 
    "PRR14L", "PTDSS1", "RAB11FIP5", "RALGAPB", "RGS7", "SATB1", "SF3B1", "SLC45A1", "SLC7A5", 
    "SLC7A7", "SLCO1B3", "SRGAP3", "SRSF11", "SYP", "TBC1D23", "TBCK", "TFE3", "TLE3", "TMEM39B", 
    "TNRC6C", "TRIM8", "TRPC4", "TRPC6", "TRPM1", "TRPM6", "TSHZ1", "TUBGCP5", "UBR3", "UIMC1", 
    "VCP", "WDR37", "WNT1", "ACAP2", "ACE", "ADGRL1", "AGBL4", "AGO4", "AMPD1", "ANP32A", 
    "AP2S1", "ARF3", "ASAP2", "ASB11", "ASB14", "ATP6V0A2", "BCAS1", "BICRA", "CADPS", "CDK8", 
    "CELF2", "CELF6", "CHMP1A", "COL28A1", "CPT2", "CPZ", "CTNNA2", "CUL2", "CUL4B", "CXXC5", 
    "DENR", "DGKI", "DHX9", "DOLK", "DNAJC5", "ECPAS", "EIF3G", "EMSY", "ENPP1", "ETFB", "EXOC6B", 
    "FAN1", "FBXO33", "FGA", "FGFR1", "FRMD5", "GBE1", "GPD2", "GRID2IP", "HLA-B", "HNRNPD", 
    "HYDIN", "INPP1", "KCNN2", "KDM5A", "KDM1A", "LEO1", "LHX2", "LNPK", "LRRC4", "MAP4K1", 
    "MAPT", "MAST1", "MSR1", "MYO7A", "NAALADL2", "OXT", "P4HA2", "PCDH11X", "PCDHA1", "PCDHA12", 
    "PCDHA13", "PCDHA3", "PCDHA8", "PCDHA9", "PCDHAC1", "PEX7", "PHF12", "PHF3", "PHRF1", 
    "PIK3CG", "PLCD4", "PLXNB1", "PON1", "POU3F3", "PPP1R9B", "PPP2CA", "PPP2R5C", "PTPN4", 
    "PUF60", "RBBP5", "RBM27", "RNF135", "RPS6KA2", "SETDB1", "SH3RF1", "SMAP2", "SMG6", "SNX5", 
    "SPARCL1", "STK39", "TAF1C", "TAF4", "THBS1", "THRA", "TM9SF4", "TRIM32", "TSPAN4", "UBAP2L", 
    "UBN2", "UGGT1", "UNC5D", "USP15", "WASF1", "YEATS2", "YWHAE", "ZFHX4", "ZNF517", "ZNF711", 
    "ZWILCH", "ABCA10", "ABCE1", "ADCY1", "ADCY3", "ADORA3", "AGO2", "APBB1", "ARHGAP11B", 
    "AZGP1", "BACE1", "BRINP3", "BTRC", "CAMK2D", "CAT", "CCSER1", "CD276", "CDON", "CLASP1", 
    "CMPK2", "CPEB4", "CX3CR1", "CYLC2", "DDX23", "CYP11B1", "DLX3", "DMWD", "DOCK1", "DPP3", 
    "DRD1", "EIF4G1", "EIF5A", "FAM98C", "FBRSL1", "FCRL6", "FGF13", "GALNT8", "H4C11", "HLA-G", 
    "HNRNPF", "HNRNPUL2", "KATNAL1", "KDM3A", "KDM4C", "KHDRBS2", "KIAA0232", "KLHL20", "KRT26", 
    "LARP1", "LDB1", "LDLR", "LEMD3", "LILRB2", "LMX1B", "MAP1A", "MAPK8IP1", "MBD3", "MCM4", 
    "MCM6", "MED23", "LZTS2", "MKX", "MLANA", "MYLK", "NCKAP5", "NCOA1", "NEDD4", "NEO1", 
    "NR1D1", "NMT1", "NPAS2", "NUDCD2", "NUP155", "OR1C1", "OR2T10", "OTX1", "PAFAH1B2", "PCDHA6", 
    "PCDHAC2", "PDCD1", "PHF7", "PLN", "POMT1", "PRKD2", "PTCHD1-AS", "PYHIN1", "RAP1A", 
    "RASSF5", "RFX7", "RICTOR", "RIMS3", "RNF25", "RUNX1T1", "SAE1", "SAMD11", "SASH1", "SCAF1", 
    "SCFD2", "SCGN", "RPS10P2-AS1", "SDC2", "SENP6", "SERPINE1", "SH3RF3", "SLC24A2", "SLC35G1", 
    "SLFN5", "SOX6", "SPP2", "SPRY2", "SRSF1", "SSR4", "ST7", "STYK1", "SYT17", "TCEAL1", 
    "TEK", "TGM1", "TM4SF19", "TM4SF20", "TNS2", "TRAP1", "TRAPPC2L", "TSPAN17", "UBR1", 
    "TDO2", "VAMP2", "VASH1", "VIL1", "VSIG4", "ZBTB21", "ZC3H11A", "ZFX", "ZNF532", "ZNF548", 
    "ZNF626", "ZNF774", "ZNF827", "ADSS2", "AMT", "APH1A", "ARHGAP30", "ATP2B1", "ATXN2", 
    "C15orf62", "CACNB1", "CAMTA2", "CBX1", "CCT4", "CAPZA2", "CBX4", "CDK16", "CECR2", "CHRNB3", 
    "CHST2", "CORO1A", "CSNK1G1", "CTNND1", "DAPP1", "CTPS1", "DIPK2A", "DNER", "DPYSL3", 
    "EGR3", "EXOC3", "EXOC6", "FABP4", "FBXL13", "FRG1", "FRK", "GABRA3", "GDA", "GGNBP2", 
    "GLIS1", "GNB1L", "GPC4", "GPR37", "GUCY1A2", "H2BC11", "H4C3", "GSTM1", "IL1RAPL2", "IRX5", 
    "KCNJ15", "KCNK7", "KDM1B", "KRR1", "LIN7B", "LRRC4C", "MBD6", "MEMO1", "MYOCD", "NASP", 
    "NIPA2", "NOVA2", "NPTN", "NDUFA5", "OR52M1", "PDE1C", "PLAUR", "POT1", "PPP5C", "PRPF19", 
    "PRPF39", "PSMC5", "PSMD11", "PSMD6", "RAB11FIP4", "RAD21L1", "REEP3", "RFX4", "RHEB", 
    "RHOXF1", "RIT2", "SCP2", "SENP1", "SETDB2", "SHOX", "SLC25A39", "SLC27A4", "SLC38A10", 
    "SLC7A3", "SMURF1", "SNCAIP", "SOD1", "SRPRA", "SSRP1", "SYCE1", "SYNJ1", "TBCEL", "TBCB", 
    "TMEM134", "TTI2", "USP30", "VEZF1", "VPS54", "VWA7", "WDR5", "ZBTB16", "ZNF385B", "ZNF644", 
    "ZNF713", "ADK", "AGAP5", "ALG6", "ANXA1", "BICDL1", "CARD11", "CCIN", "CCNG1", "CIB2", 
    "CLIP2", "DYDC1", "ELOVL2", "ENOX2", "EXOC5", "FAM47A", "FGF14", "FRRS1L", "FXN", "GATM", 
    "GFAP", "HLA-DPB1", "HTR2C", "HNRNPL", "KCNA3", "KIZ", "KNG1", "MAP4K4", "METTL26", 
    "MSANTD2", "MSX2", "NKX2-2", "NXF1", "PEBP4", "PLEKHA8", "PPP1R1B", "PRR25", "RAB43", 
    "RNF38", "RNU2-2", "RNU5B-1", "SLC29A4", "SYAP1", "TBX22", "TPO", "TRIM33", "UBE2H", 
    "USP9Y", "WWP1", "XRCC6", "ZBTB47", "ZBTB7A", "ZNF18", "ASB9", "CD99L2", "DYDC2", "ERG", 
    "ERMN", "GAS2", "HMGN1", "KLF16", "LEP", "MAPT-AS1", "NCAPH2", "MNT", "OR2M4", "PHB1", 
    "PJA1", "RP11-1407O15.2", "SLC22A15", "SLC25A27", "SLC35B1", "SYBU", "TBL1X", "TERB2", 
    "TFB2M"
}

# ========================================================================
# 3. BUILD INTERVALS 
# ========================================================================
print("Downloading Gene Map to filter intervals...")
gencode_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.basic.annotation.gtf.gz"
gene_map_df = pd.read_csv(gencode_url, sep='\t', comment='#', header=None, names=['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes'])

def extract_gene_name(attr):
    if 'gene_name "' in attr:
        return attr.split('gene_name "')[1].split('"')[0]
    return None

gene_map_df = gene_map_df[gene_map_df['feature'] == 'gene']
gene_map_df['gene_name'] = gene_map_df['attributes'].apply(extract_gene_name)
target_intervals_df = gene_map_df[gene_map_df['gene_name'].isin(asd_gene_target_list)]

intervals = []
for _, row in target_intervals_df.iterrows():
    chrom = str(row['chrom']).replace("chr", "")
    final_chrom = f"chr{chrom}"
    intervals.append(hl.parse_locus_interval(f"[{final_chrom}:{row['start']}-{row['end']}]"))

In [5]:
# ========================================================================
# 4. LOAD & FILTER
# ========================================================================
print("Reading MT with Interval Filter...")
mt = hl.read_matrix_table(wgs_path)

if intervals:
    # This is the speed hack
    mt = hl.filter_intervals(mt, intervals)
    print("✓ Speed Hack Applied.")
else:
    print("Warning: No intervals found.")


Reading MT with Interval Filter...
✓ Speed Hack Applied.


In [ ]:

# ========================================================================
# 5. FILTER COHORTS (10 PEOPLE)
# ========================================================================
if 'dataset_1_person_df' in locals():
    asd_ids_pool = dataset_63330128_person_df['person_id'].astype(str).tolist()
else:
    asd_ids_pool = [] 

if 'dataset_2_person_df' in locals():
    control_ids_pool = dataset_84568391_person_df['person_id'].astype(str).tolist()
else:
    control_ids_pool = []

if asd_ids_pool and control_ids_pool:
    n_select = 5
    selected_asd_ids = random.sample(asd_ids_pool, min(len(asd_ids_pool), n_select))
    selected_control_ids = random.sample(control_ids_pool, min(len(control_ids_pool), n_select))
    all_sample_ids = selected_asd_ids + selected_control_ids
    
    print(f"Filtering to 10 Individuals: {all_sample_ids}")
    mt = mt.filter_cols(hl.literal(all_sample_ids).contains(mt.s))
else:
    print("Warning: Cohort IDs not found. Skipping sample filter.")


In [7]:
# ========================================================================
# 6. EXPORT
# ========================================================================
print("Exporting Variants...")
rows_ht = mt.rows()

variants_table = rows_ht.select(
    Chrom = rows_ht.locus.contig,
    Pos = rows_ht.locus.position,
    Ref = rows_ht.alleles[0],
    Alt = rows_ht.alleles[1],
    Gene = hl.literal("Lookup_Required")
)

variants_filename = f"{bucket}/data/aou_sfari_variants_fast.csv"
variants_table.export(variants_filename, delimiter=',')
print(f"✓ Variants Exported to {variants_filename}")

print("Exporting Genotypes...")
entries_ht = mt.entries()

entries_table = entries_ht.select(
    Var_ID = hl.str(entries_ht.locus) + "_" + entries_ht.alleles[0] + ">" + entries_ht.alleles[1],
    Sample_ID = entries_ht.s,
    GT_num = hl.if_else(hl.is_defined(entries_ht.GT), entries_ht.GT.n_alt_alleles(), 0)
)

genotypes_filename = f"{bucket}/data/aou_sfari_genotypes_long.csv"
entries_table.export(genotypes_filename, delimiter=',')
print(f"✓ Genotypes Exported to {genotypes_filename}")

print("\nSUCCESS! You are done with extraction.")

Exporting Variants...


✓ Variants Exported to gs://fc-secure-b8cbf1c7-c51b-44be-b4f7-4f88082bda65/data/aou_sfari_variants_fast.csv
Exporting Genotypes...


✓ Genotypes Exported to gs://fc-secure-b8cbf1c7-c51b-44be-b4f7-4f88082bda65/data/aou_sfari_genotypes_long.csv

SUCCESS! You are done with extraction.


In [ ]:
!gsutil ls $WORKSPACE_BUCKET/data

In [ ]:
# patient-level per person gene module score and heatmap 

import pandas as pd
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 1. SETUP & LOAD DATA
# --------------------
bucket = os.getenv("WORKSPACE_BUCKET")
print("Loading Patient Data...")

# Ensure we have the variant & genotype data
if not os.path.exists("aou_sfari_variants_fast.csv"):
    os.system(f"gsutil cp {bucket}/data/aou_sfari_variants_fast.csv .")
    os.system(f"gsutil cp {bucket}/data/aou_sfari_genotypes_long.csv .")
    os.system(f"gsutil cp {bucket}/results/final_patient_results.csv .") # Scores from Phase 2

variants_df = pd.read_csv("aou_sfari_variants_fast.csv")
genotypes_df = pd.read_csv("aou_sfari_genotypes_long.csv")

# We need the calculated risk scores from Phase 2 (re-calculating merge if needed)
# Ideally, Phase 2 added 'risk_score' to variants_df. 
# If that dataframe is lost in memory, let's assume we need to reload or re-merge.
# For safety, let's assume you ran Phase 2 and variants_df HAS 'risk_score'.
# If not, we re-load the saved result if available, or just re-calculate quickly.
# (Here I will assume variants_df has the columns: Chrom, Pos, risk_score. 
#  If 'risk_score' is missing, please re-run Phase 2 or let me know).

# 2. RE-ANNOTATE GENES (Map Chr:Pos -> Gene Symbol)
# --------------------
print("Mapping Variants to Gene Symbols...")

# Download GENCODE again (fastest way to map)
gencode_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.basic.annotation.gtf.gz"
gene_map_df = pd.read_csv(gencode_url, sep='\t', comment='#', header=None, names=['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes'])

def extract_gene_name(attr):
    if 'gene_name "' in attr:
        return attr.split('gene_name "')[1].split('"')[0]
    return None

# Filter to Genes only
gene_map = gene_map_df[gene_map_df['feature'] == 'gene'].copy()
gene_map['gene_name'] = gene_map['attributes'].apply(extract_gene_name)
gene_map = gene_map[['chrom', 'start', 'end', 'gene_name']]

# Create an Interval Index for super-fast lookup
# We use pandas IntervalIndex to find which gene range a variant falls into
# Note: This ignores overlapping genes for simplicity (taking the first match)
import pandas as pd

# Clean chrom names
gene_map['chrom'] = gene_map['chrom'].astype(str)
# Handle prefix matching
if not str(variants_df['Chrom'].iloc[0]).startswith('chr'):
    gene_map['chrom'] = gene_map['chrom'].str.replace('chr', '')

# Function to map variant to gene
# This simple loop is robust for small datasets (~1000 variants). 
# For millions, we'd use IntervalIndex.
def map_variant_to_gene(chrom, pos):
    # Find genes on this chromosome where pos is between start and end
    # We filter gene_map down to just the relevant chrom first
    candidates = gene_map[
        (gene_map['chrom'] == str(chrom)) & 
        (gene_map['start'] <= pos) & 
        (gene_map['end'] >= pos)
    ]
    if not candidates.empty:
        return candidates.iloc[0]['gene_name']
    return "Unknown"

tqdm.pandas(desc="Annotating Genes")
variants_df['Gene_Symbol'] = variants_df.apply(lambda x: map_variant_to_gene(x['Chrom'], x['Pos']), axis=1)

print(f"Annotated {variants_df['Gene_Symbol'].nunique()} unique genes.")

# 3. LOAD PATHWAY DATABASE (Enrichr GMT)
# --------------------
print("Downloading Gene Ontology (GO) Biological Process...")
# Standard GO Biological Process 2023 from Enrichr
gmt_url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2023"
response = requests.get(gmt_url)

# Parse GMT: PathwayName \t Description \t Gene1 \t Gene2 ...
pathway_dict = {}
for line in response.text.splitlines():
    parts = line.split('\t')
    pathway_name = parts[0]
    genes = set(parts[2:]) # Skip description
    pathway_dict[pathway_name] = genes

print(f"Loaded {len(pathway_dict)} pathways.")

# Filter to only relevant pathways (ASD-related keywords)
# This reduces noise and processing time
relevant_keywords = ['synap', 'neuron', 'axon', 'dendri', 'ion channel', 'chromatin', 'behavior', 'learning']
selected_pathways = {k: v for k, v in pathway_dict.items() if any(w in k.lower() for w in relevant_keywords)}
print(f"Filtered to {len(selected_pathways)} neuro-relevant pathways.")

# 4. CALCULATE SCORES PER PATHWAY
# --------------------
print("Calculating Scores per Patient per Pathway...")

# Prepare the "Patient-Gene" Table first
# We aggregate risk at the gene level for each patient
variants_df['Var_ID'] = variants_df['Chrom'].astype(str) + ":" + variants_df['Pos'].astype(str) + "_" + variants_df['Ref'] + ">" + variants_df['Alt']
merged = genotypes_df.merge(variants_df[['Var_ID', 'Gene_Symbol', 'risk_score']], on='Var_ID')
merged = merged[merged['GT_num'] > 0] # Only actual mutations
merged['burden'] = merged['GT_num'] * merged['risk_score']

# Sum scores per Gene per Patient
patient_gene_scores = merged.groupby(['Sample_ID', 'Gene_Symbol'])['burden'].sum().reset_index()

# Now aggregate into Pathways
patient_pathway_scores = []

for patient_id in patient_gene_scores['Sample_ID'].unique():
    pat_data = patient_gene_scores[patient_gene_scores['Sample_ID'] == patient_id]
    
    # Create a set of this patient's mutated genes for fast lookup
    pat_genes = dict(zip(pat_data['Gene_Symbol'], pat_data['burden']))
    
    for path_name, path_genes in selected_pathways.items():
        # Score = Sum of burdens of genes in this pathway
        score = 0
        overlap_count = 0
        for gene in path_genes:
            if gene in pat_genes:
                score += pat_genes[gene]
                overlap_count += 1
        
        if score > 0: # Only store non-zero scores
            patient_pathway_scores.append({
                'Person_ID': patient_id,
                'Pathway': path_name.split(' (GO')[0], # Clean name
                'Pathway_Score': score,
                'Gene_Count': overlap_count
            })

pathway_df = pd.DataFrame(patient_pathway_scores)

# 5. VISUALIZATION (Heatmap)
# --------------------
if not pathway_df.empty:
    # Pivot for Heatmap: Rows=Pathway, Cols=Patient
    matrix = pathway_df.pivot_table(index='Pathway', columns='Person_ID', values='Pathway_Score', fill_value=0)
    
    # Filter for top active pathways (e.g., top 20 by max score) to keep plot readable
    top_pathways = matrix.max(axis=1).sort_values(ascending=False).head(20).index
    matrix_top = matrix.loc[top_pathways]

    plt.figure(figsize=(12, 10))
    sns.heatmap(matrix_top, cmap="Reds", annot=True, fmt=".1f", linewidths=.5)
    plt.title("Pathogenicity Score by Biological Pathway per Patient")
    plt.xlabel("Patient ID")
    plt.ylabel("GO Biological Process")
    plt.tight_layout()
    plt.savefig("patient_pathway_heatmap.png")
    plt.show()
    
    # Save Results
    os.system(f"gsutil cp patient_pathway_heatmap.png {bucket}/results/")
    pathway_df.to_csv("final_patient_pathway_scores.csv", index=False)
    os.system(f"gsutil cp final_patient_pathway_scores.csv {bucket}/results/")
    print(f"\n✓ Saved Pathway Scores to {bucket}/results/final_patient_pathway_scores.csv")
    
    # 
else:
    print("No non-zero pathway scores found (Did the patients have variants in the gene list?)")